In [ ]:
import pandas as pd
import sys
sys.path.insert(0, '../')
from cleaner import Cleaner, Corretor
from tqdm.auto import tqdm
from pandarallel import pandarallel

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/LipSet/LipSet_chunk.csv')
dados

In [ ]:
lista_textos = []
for indice in tqdm(dados.index):
    if isinstance(dados.loc[indice, 'text'], str):
        textos = dados.loc[indice, 'text'].split('\n')
        lista_textos.extend(textos)
dados = pd.DataFrame({'text': lista_textos})
dados

In [ ]:
dados = dados.select_dtypes(include=['object'])
dados.dropna(inplace=True)
dados.drop_duplicates(inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados['tamanho'] = dados['text'].apply(lambda x: len(x.split()))
dados = dados[dados['tamanho'] > 10]
dados

In [ ]:
dados['tamanho'].describe()

In [ ]:
dados.drop(columns=['tamanho'], inplace=True)
dados.dropna(inplace=True)
dados.drop_duplicates(inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased, accents) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    pandarallel.initialize(progress_bar=True)
    dataframe[column] = dataframe[column].parallel_apply(lambda x: Cleaner().clear(x))
    dataframe[column] = dataframe[column].parallel_apply(lambda x: Corretor(cased, accents).corrige_termos(x))
    return dataframe

In [ ]:
dados = executaLimpeza(dados, 'text', cased=False, accents=False)
dados

In [ ]:
dados = dados.select_dtypes(include=['object'])
dados.dropna(inplace=True)
dados.drop_duplicates(inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Modelos/PreTreinamento/HelBERT-uncased-fs/checkpoint-epoca-6')

In [ ]:
tqdm.pandas()
dados['tamanho'] = dados['text'].progress_apply(lambda x: len(tokenizer(x)['input_ids']))

In [ ]:
dados['tamanho'].describe()

In [ ]:
dados = dados[dados['tamanho'] <= 128]
dados = dados[dados['tamanho'] >= 10]
dados

In [ ]:
dados.drop(columns=['tamanho'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/LipSet/LipSet_chunk_lowercase.csv', index=False)